In [2]:
%cd "/home/kera/workspace/Transformer-GB"

/data/kera/workspace/Transformer-GB


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import yaml 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import argparse
import re 
import numpy as np
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoConfig

from transformers import PreTrainedTokenizerFast
from src.loader.data import load_data
from src.loader.checkpoint import load_trained_bag
from src.evalution.evaluators import eval_prediction

load('src/data/symbolic_utils.sage')
load('src/data/gbdataset.sage')

-f


In [5]:
def sol_listform(sol):
    return list(sol.values())[::-1]

def print_solution(sols, ring):
    for s in sols:
        print([s[x] for x in ring.gens()])

In [6]:
def coordinate_change_map(ring, num_bound=None):
    while True:
        n = ring.ngens()
        if num_bound is not None:
            P = matrix.random(ring.base_ring(), n, n, num_bound=num_bound)
        else:
            P = matrix.random(ring.base_ring(), n, n)
            
        if P.rank()  == n: break

    gens = ring.gens()
    trans_gens = matrix(gens) * P
    trans_gens = trans_gens[0]
    change_map = dict(zip(gens, trans_gens))
    return change_map

def coordinate_change(F, change_map):
    return [f.subs(change_map) for f in F]

def is_shape_position(G, verbose=False, relaxed_lt_condition=False):
    ring = G.ring()
    gens = ring.gens()

    # if verbose:
    #     print('Input polynomial set:')
    #     print('---------------------------')
    #     for i, g in enumerate(G): print(f' g_{i} = {g}')
    #     print('---------------------------')

    if not G.is_groebner(): 
        if verbose: print(f'[GB Error] Given G is not a GB.')
        return False

    xn = gens[-1]
    s = len(G)

    univariate_check = [G[-1].variables() == (xn,)]
    if not (univariate_check): 
        if verbose: print(f'[Univariaticity Error] Last polynomial is not univariate in core variable. \n {G[-1]}')
        return False
    
    if relaxed_lt_condition:
        lt_check = [g.lt().variables() == (x,) for g, x in zip(G[:-1], gens[:-1])]
    else:
        lt_check = [g.lt() == x for g, x in zip(G[:-1], gens[:-1])]
    if not all(lt_check): 
        if verbose: 
            if relaxed_lt_condition:
                print(f"[Leading Term Error (relaxed)] First {s-1} polynomials are not all led by x_i^e")
            else:
                print(f"[Leading Term Error ] First {s-1} polynomials are not all led by x_i")
            for i, (g, success) in enumerate(zip(G[:-1], lt_check)):
                if success:
                    print(f' [ok] g_{i} = {g}')
                else:
                    print(f' [  ] g_{i} = {g}')

        return False
    
    difference_form_check = [(g.lm() - g).variables() in ((xn,), ()) for g in G[:-1]]
    if not all(difference_form_check): 
        if verbose: 
            print(f'[Difference form Error] First {s-1} polynomials are not all x_i - h({xn}) form.')
            for i, (g, success) in enumerate(zip(G[:-1], difference_form_check)):
                if success:
                    print(f' [ok] g_{i} = {g}')
                else:
                    print(f' [  ] g_{i} = {g}')
        return False

    return True


In [245]:
ring = PolynomialRing(QQ, 'x', 2, order='lex')
x1, x2 = ring.gens()

In [246]:
p1 = ring(x1 - x2 + 1)
p2 = ring((-x1-2*x2)^2 + (2*x1 - 3*x2)^2 + (4*x2)^2 - 8^2 - (-5)^2 - (-4)^2)
F = [p1, p2]

In [247]:
I = ideal(F)
G = I.groebner_basis()
sols = I.variety()
sols

[{x1: 2, x0: 1}, {x1: -25/13, x0: -38/13}]

In [248]:
is_shape_position(G, verbose=True)

True

In [443]:
def shuffled_least_square_demo(n=3, num_points=6, base_field=QQ, num_symmetric_polys=4, num_bound=3):
    ring = PolynomialRing(base_field, 'x', n, order='lex')
    point_ring = PolynomialRing(base_field, 'z', num_points, order='lex')

    xs = ring.gens()
    zs = point_ring.gens()

    # prepare symmetric polynomials
    # symmetric_polys = [sum(zs),
    #                    sum([z^2 for z in zs]),
    #                    sum([z1*z2 for z1, z2 in zip(zs, zs[1:]+zs[:1])]),
    #                    sum([z^3 for z in zs]),
    #                    sum([z1*z2*z3 for z1, z2, z3 in zip(zs, zs[1:]+zs[:1], zs[2:]+zs[:2])]),
    #                    ]
    symmetric_polys = [sum([z^i for z in zs]) for i in range(1, 2*num_symmetric_polys+1)]
    symmetric_polys =  symmetric_polys[:num_symmetric_polys]
    symmetric_polys = ideal(symmetric_polys).basis
    print('-- Symmetric polynomials ------------')
    for i, sp in enumerate(symmetric_polys): print(f's_{i} = {sp}')
    print('')

    # setup the problem
    kwargs = {'num_bound': num_bound} if base_field == QQ else {}
    A = matrix.random(ring.base_ring(), num_points, n, **kwargs)
    v = matrix.random(ring.base_ring(), n, 1, **kwargs)
    y = A * v
    P = random_permutation_matrix(num_points)
    print('-- Problem setup (Av = Py) ----------')
    print(f'A \n{A}')
    print(f"y' = {y.T}")
    print(f"v' = {v.T}")
    print(f'P \n{P}')
    print('')

    F = []
    for sp in symmetric_polys:
        f1 = sp(*vector(A * matrix(xs).T))
        f2 = sp(*vector((P * y).T))
        F.append(f1 - f2)

    I = ideal(F)
    G = I.groebner_basis()
    print('-- Original system ---------------')
    for i, f in enumerate(F): print(f'f_{i} = {f}')
    print('')

    print('-- Gröbner basis -----------------')
    for i, g in enumerate(G): print(f'g_{i} = {g}')

    is_shape = is_shape_position(G, verbose=True)
    print(f'[{is_shape}] <G> is in shape position')
    print('')

    dim = I.dimension()
    print(f'The ideal has dimension {dim}.')
    if dim != 0: 
        print('Thus, no solution exists.')

    else:        
        sols = I.variety()
        print(f'{len(sols)} solutions found.')
        print_solution(sols, ring)

        print(f'true solution is ')
        print(vector(v.T))

    output = {'A': A, 'v': v, 'y': y, 'P': P, 'F': F, 'G': G, 'sols': sols, 'ring': ring, 'symmetric_polys': symmetric_polys, 'is_shape': is_shape, 'num_bound': num_bound}

    return output
        

In [492]:
output = shuffled_least_square_demo(n=4, 
                                    num_points=6, 
                                    base_field=QQ, 
                                    num_symmetric_polys=5, 
                                    num_bound=3)

-- Symmetric polynomials ------------
s_0 = z0 + z1 + z2 + z3 + z4 + z5
s_1 = z0^2 + z1^2 + z2^2 + z3^2 + z4^2 + z5^2
s_2 = z0^3 + z1^3 + z2^3 + z3^3 + z4^3 + z5^3
s_3 = z0^4 + z1^4 + z2^4 + z3^4 + z4^4 + z5^4
s_4 = z0^5 + z1^5 + z2^5 + z3^5 + z4^5 + z5^5

-- Problem setup (Av = Py) ----------
A 
[ 1/2   -2    0 -1/2]
[   0    0   -3    0]
[   0    3   -3    0]
[  -3   -1    0    3]
[ 1/2   -3   -3  3/2]
[  -2    0 -1/2  1/2]
y' = [  5/2    -3  -9/2 -17/2     0 -13/2]
v' = [   3 -1/2    1    0]
P 
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 0 1]
[0 1 0 0 0 0]
[0 0 0 0 1 0]
[1 0 0 0 0 0]

-- Original system ---------------
f_0 = -4*x0 - 3*x1 - 19/2*x2 + 9/2*x3 + 20
f_1 = 27/2*x0^2 + x0*x1 - x0*x2 - 19*x0*x3 + 23*x1^2 - 13*x1*x3 + 109/4*x2^2 - 19/2*x2*x3 + 47/4*x3^2 - 150
f_2 = -139/4*x0^3 - 123/4*x0^2*x1 - 33/4*x0^2*x2 + 351/4*x0^2*x3 + 21/2*x0*x1^2 + 27*x0*x1*x2 + 87/2*x0*x1*x3 + 12*x0*x2^2 - 21/2*x0*x2*x3 - 315/4*x0*x3^2 - 9*x1^3 - 162*x1^2*x2 + 87/2*x1^2*x3 + 81*x1*x2*x3 - 195/4*x1*x3^2 - 6

In [493]:
def load_model_and_tokenizer(n, field_name):    
    
    load_dir = f'results/shape_gb_lex/gb_dataset_n={n}_field={field_name}'
    print(f'-- Loading model from {load_dir} ---------')

    bag = load_trained_bag(load_dir, from_checkpoint=True)
    model = bag['model'] 
    tokenizer = bag['tokenizer']
    params = bag['params']

    return model, tokenizer

In [494]:
def Transformer_GB_demo(model, tokenizer, F, G, ring, num_beams=1):
    F_prefix = [poly_to_prefix(f) for f in F]
    G_prefix = [poly_to_prefix(g) for g in G]
    x_text = ' [SEP] '.join(F_prefix)
    y_text = ' [SEP] '.join(G_prefix)

    x = tokenizer(x_text, return_tensors='pt')['input_ids'].cuda()
    # y = tokenizer(y_text, return_tensors='pt')['input_ids'].cuda()
    output_ids = model.generate(x, max_length=1000, num_beams=num_beams, do_sample=False)
    z_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    G_pred = [prefix_to_poly(zt, ring) for zt in z_text[0].split('[SEP]')]

    print('-- Gröbner basis (answer) ------------')
    for i, g in enumerate(G): print(f'g_{i} = {g}')
    print('')

    print('-- Gröbner basis (Transformer) -------')
    for i, g in enumerate(G_pred): print(f'g_{i} = {g}')
    print('')

    sols = ideal(G_pred).variety()
    print(f'{len(sols)} solutions found.')
    print_solution(sols, ring)

レイノルズ作用素を使えば次数があがらないか

In [511]:
n = 3
field = QQ
field_name = 'QQ'
num_bound = 100

output = shuffled_least_square_demo(n=n,
                                    num_points=8, 
                                    base_field=field, 
                                    num_symmetric_polys=n, 
                                    num_bound=num_bound)

-- Symmetric polynomials ------------
s_0 = z0 + z1 + z2 + z3 + z4 + z5 + z6 + z7
s_1 = z0^2 + z1^2 + z2^2 + z3^2 + z4^2 + z5^2 + z6^2 + z7^2
s_2 = z0^3 + z1^3 + z2^3 + z3^3 + z4^3 + z5^3 + z6^3 + z7^3

-- Problem setup (Av = Py) ----------
A 
[   7  -99  -10]
[  12  -71   29]
[21/2   82  -53]
[ -41   -5    8]
[  63   17   73]
[  92   45  -19]
[ -41 47/2   99]
[ -43   56  -15]
y' = [   2703    4560   -6174     -21    5610   -1140 10635/2   -3657]
v' = [ 18 -33  69]
P 
[1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 1 0 0 0]

-- Original system ---------------
f_0 = 119/2*x0 + 97/2*x1 + 112*x2 - 14397/2
f_1 = 71789/4*x0^2 + 2721*x0*x1 - 2339*x0*x2 + 110373/4*x1^2 - 7165*x1*x2 + 19530*x2^2 - 562558725/4
f_2 = 6516917/8*x0^3 + 1730883*x0^2*x1 + 3346749/4*x0^2*x2 + 2948313/4*x0*x1^2 - 729087*x0*x1*x2 - 29487/2*x0*x2^2 - 3938681/8*x1^3 - 3812859/4*x1^2*x2 + 3061011/2*x1*x2^2 + 1224106*x2^3 - 1246010847867

In [7]:
float(1382839156933011595142027797967880468915999/4899595278744365131208409279962465757274)

282.23538440656597

In [10]:
QQ(282.23538440656597)

5279484658/18705963

In [9]:
float(QQ(282.23538440656597))

282.23538440656597

In [502]:
model, tokenizer = load_model_and_tokenizer(n, field_name)
F, G, ring, v = output['F'], output['G'], output['ring'], output['v']
Transformer_GB_demo(model, tokenizer, F, G, ring, num_beams=1)
print(f'(True solution is {v.T})')

-- Loading model from results/shape_gb_lex/gb_dataset_n=3_field=QQ ---------


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


Exception: WordLevel error: Missing [UNK] token from the vocabulary